In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from sklearn.metrics import classification_report, accuracy_score

# Load and preprocess the dataset
df = pd.read_csv('C:\\Users\\Shaurya\\Downloads\\data.csv')
df = df.drop(['id', 'Unnamed: 32'], axis=1)
X = df.drop('diagnosis', axis=1)
y = df['diagnosis'].apply(lambda x: 1 if x == 'M' else 0)
y = to_categorical(y)

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Pad the data to get 36 features (6x6)
X_padded = np.pad(X_scaled, ((0, 0), (0, 36 - X_scaled.shape[1])), 'constant')

# Reshape the data to 6x6 and then pad it to 32x32
X_reshaped = np.reshape(X_padded, (-1, 6, 6))
X_padded = np.pad(X_reshaped, ((0, 0), (0, 26), (0, 26)), 'constant')

# Add an additional dimension to mimic 3 channels (RGB)
X_padded = np.stack((X_padded,)*3, axis=-1)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.2, random_state=42)

# Load the MobileNetV2 model
base_model = MobileNetV2(include_top=False, input_shape=(32, 32, 3), pooling='avg')
base_model.trainable = False

# Build and compile the model
model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(2, activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

9406464/9406464 [==============================] - 1s 0us/step
Epoch 1/10
15/15 [==============================] - 7s 169ms/step - loss: 0.6866 - accuracy: 0.5736 - val_loss: 0.6846 - val_accuracy: 0.6140
Epoch 2/10
15/15 [==============================] - 1s 56ms/step - loss: 0.6796 - accuracy: 0.6286 - val_loss: 0.6784 - val_accuracy: 0.6228
Epoch 3/10
15/15 [==============================] - 1s 57ms/step - loss: 0.6737 - accuracy: 0.6286 - val_loss: 0.6731 - val_accuracy: 0.6228
Epoch 4/10
15/15 [==============================] - 1s 58ms/step - loss: 0.6687 - accuracy: 0.6286 - val_loss: 0.6690 - val_accuracy: 0.6228
Epoch 5/10
15/15 [==============================] - 1s 53ms/step - loss: 0.6645 - accuracy: 0.6286 - val_loss: 0.6647 - val_accuracy: 0.6228
Epoch 6/10
15/15 [==============================] - 1s 54ms/step - loss: 0.6598 - accuracy: 0.6286 - val_loss: 0.6609 - val_accuracy: 0.6228
Epoch 7/10
15/15 [==============================] - 1s 57ms/step - loss: 0.6559 - accuracy

In [2]:
# Evaluate the model
y_pred_prob = model.predict(X_test)
y_pred = np.argmax(y_pred_prob, axis=1)
y_true = np.argmax(y_test, axis=1)
accuracy = accuracy_score(y_true, y_pred)
report = classification_report(y_true, y_pred)

# Print the results
print(f"Accuracy: {accuracy:.4f}")
print("Classification Report:\n", report)

4/4 [==============================] - 1s 34ms/step
Accuracy: 0.6228
Classification Report:
               precision    recall  f1-score   support

           0       0.62      1.00      0.77        71
           1       0.00      0.00      0.00        43

    accuracy                           0.62       114
   macro avg       0.31      0.50      0.38       114
weighted avg       0.39      0.62      0.48       114



C:\Users\Shaurya\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Shaurya\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Shaurya\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
